# Airbnb Data Exploration Notebook
_A Fall 2025 Team Project_

## 1. File Format Overview


This project uses multiple file formats, each with tradeoffs:

### ✅ CSV (Comma-Separated Values)
- Human-readable, plaintext format.
- No type information (everything is read as strings unless inferred).
- Large file size, no compression.

### ✅ TSV (Tab-Separated Values)
- Same as CSV but uses tabs instead of commas.
- Used here for metadata description files.

### ✅ Parquet
- Binary, columnar format optimized for analytics.
- Built-in support for compression (e.g., Snappy).
- Preserves data types and schema.
- Excellent with Spark, DuckDB, and modern data tools.


## 2. Tools for Data Manipulation and Exploration


We will use three main tools:

### ✅ Pandas
- Python-native
- Best for local, small-to-medium datasets
- Simple and expressive syntax

### ✅ PySpark
- Distributed data processing engine (Apache Spark)
- Best for large-scale data (millions of rows)
- Lazy evaluation and optimized execution

### ✅ SQL (via DuckDB or Spark SQL)
- Universal language for querying structured data
- Easily integrates with both Pandas and Spark
- Great for expressing aggregations and joins


## 3. Imports and Setup

In [ ]:

import pandas as pd
from pyspark.sql import SparkSession
import duckdb

# Start Spark
spark = SparkSession.builder.appName("AirbnbExploration").getOrCreate()


## 4. Load NYC Listings Dataset

In [ ]:

# Load parquet using pandas
df_pd = pd.read_parquet("../data/nyc/nyc-listings.parquet")
df_pd.head()


In [ ]:

# Load parquet using PySpark
df_spark = spark.read.parquet("../data/nyc/nyc-listings.parquet")
df_spark.show(5)


In [ ]:

# Load parquet using DuckDB SQL
duckdb.sql("CREATE TABLE listings AS SELECT * FROM '../data/nyc/nyc-listings.parquet'")
duckdb.sql("SELECT * FROM listings LIMIT 5")


## 5. Initial Exploration Questions

### Q1: How many rows and columns are in the dataset?

In [ ]:

# Pandas
df_pd.shape


In [ ]:

# PySpark
df_spark.count(), len(df_spark.columns)


In [ ]:

# SQL (DuckDB)
duckdb.sql("SELECT COUNT(*) AS rows, COUNT(*) OVER () AS cols FROM listings LIMIT 1")


### Q2: What are the column names and data types?

In [ ]:
df_pd.dtypes

In [ ]:
df_spark.printSchema()

In [ ]:
duckdb.sql('PRAGMA table_info(listings)')

### Q3: How many listings per room type?

In [ ]:
df_pd['room_type'].value_counts()

In [ ]:

df_spark.groupBy("room_type").count().orderBy("count", ascending=False).show()


In [ ]:

duckdb.sql("SELECT room_type, COUNT(*) AS count FROM listings GROUP BY room_type ORDER BY count DESC")


### Q4: What are the average and median prices?

In [ ]:
df_pd['price'].describe()[['mean', '50%']]

In [ ]:

from pyspark.sql.functions import avg, expr
df_spark.select(avg("price"), expr("percentile(price, 0.5)")).show()


In [ ]:

duckdb.sql("SELECT AVG(price), MEDIAN(price) FROM listings")


## 6. Deeper Data Exploration (Exercises)

### Q5: Which columns have missing values, and how many?

In [ ]:
df_pd.isnull().sum().sort_values(ascending=False).head(10)

In [ ]:

from pyspark.sql.functions import col, isnan, when, count
df_spark.select([count(when(col(c).isNull(), c)).alias(c) for c in df_spark.columns]).show()


In [ ]:

duckdb.sql("""
SELECT column_name, COUNT(*) AS null_count
FROM (
  SELECT * FROM listings
) t
UNPIVOT (value FOR column_name IN (*))
WHERE value IS NULL
GROUP BY column_name
ORDER BY null_count DESC
LIMIT 10
""")


### Q6: How many listings are there in each neighborhood?

In [ ]:
df_pd['neighbourhood_cleansed'].value_counts().head(10)

In [ ]:

df_spark.groupBy("neighbourhood_cleansed").count().orderBy("count", ascending=False).show(10)


In [ ]:

duckdb.sql("SELECT neighbourhood_cleansed, COUNT(*) AS count FROM listings GROUP BY 1 ORDER BY 2 DESC LIMIT 10")


### Q7: What is the distribution of `availability_365`?

In [ ]:
df_pd['availability_365'].hist()

In [ ]:

df_spark.groupBy("availability_365").count().orderBy("availability_365").show(10)


In [ ]:

duckdb.sql("SELECT availability_365, COUNT(*) FROM listings GROUP BY availability_365 ORDER BY availability_365 LIMIT 10")


### Q8: How many listings are available for most of the year (availability_365 > 300)?

In [ ]:
(df_pd['availability_365'] > 300).sum()

In [ ]:

df_spark.filter("availability_365 > 300").count()


In [ ]:

duckdb.sql("SELECT COUNT(*) FROM listings WHERE availability_365 > 300")


### Q9: How many listings have zero reviews?

In [ ]:
(df_pd['number_of_reviews'] == 0).sum()

In [ ]:

df_spark.filter("number_of_reviews = 0").count()


In [ ]:

duckdb.sql("SELECT COUNT(*) FROM listings WHERE number_of_reviews = 0")


### Q10: Who are the top 10 hosts by number of listings?

In [ ]:
df_pd['host_id'].value_counts().head(10)

In [ ]:

df_spark.groupBy("host_id").count().orderBy("count", ascending=False).show(10)


In [ ]:

duckdb.sql("SELECT host_id, COUNT(*) AS count FROM listings GROUP BY 1 ORDER BY 2 DESC LIMIT 10")


### Q11: What's the average price for listings with different minimum night requirements?

In [ ]:

df_pd.groupby("minimum_nights")["price"].mean().sort_values(ascending=False).head(10)


In [ ]:

df_spark.groupBy("minimum_nights").avg("price").orderBy("avg(price)", ascending=False).show(10)


In [ ]:

duckdb.sql("SELECT minimum_nights, AVG(price) FROM listings GROUP BY minimum_nights ORDER BY AVG(price) DESC LIMIT 10")


### Q12: Are there listings with suspiciously high prices (e.g., > $1000)?

In [ ]:
df_pd[df_pd['price'] > 1000][['name', 'price']].head()

In [ ]:

df_spark.filter("price > 1000").select("name", "price").show(5)


In [ ]:

duckdb.sql("SELECT name, price FROM listings WHERE price > 1000 LIMIT 5")


### Q13: What's the average number of reviews per month by room type?

In [ ]:
df_pd.groupby('room_type')['reviews_per_month'].mean()

In [ ]:

df_spark.groupBy("room_type").avg("reviews_per_month").show()


In [ ]:

duckdb.sql("SELECT room_type, AVG(reviews_per_month) FROM listings GROUP BY room_type")


### Q14: What are the most common values for `minimum_nights`?

In [ ]:
df_pd['minimum_nights'].value_counts().head(10)

In [ ]:

df_spark.groupBy("minimum_nights").count().orderBy("count", ascending=False).show(10)


In [ ]:

duckdb.sql("SELECT minimum_nights, COUNT(*) FROM listings GROUP BY minimum_nights ORDER BY COUNT(*) DESC LIMIT 10")


### Q15: Which neighborhoods have the highest average price?

In [ ]:

df_pd.groupby("neighbourhood_cleansed")["price"].mean().sort_values(ascending=False).head(10)


In [ ]:

df_spark.groupBy("neighbourhood_cleansed").avg("price").orderBy("avg(price)", ascending=False).show(10)


In [ ]:

duckdb.sql("SELECT neighbourhood_cleansed, AVG(price) FROM listings GROUP BY 1 ORDER BY 2 DESC LIMIT 10")


## 7. Advanced Queries and Joins


We'll now perform some more complex operations by joining the `listings` dataset with the `reviews` dataset.

This will allow us to explore relationships between listings and their reviews — for example:

- Average review scores by neighborhood
- Number of reviews per listing
- Listings with unusually high/low review scores


### Step 1: Load the Reviews Dataset

In [ ]:

# Load reviews using pandas
df_reviews_pd = pd.read_parquet("../data/nyc/nyc-reviews.parquet")
df_reviews_pd.head()


In [ ]:

# Load reviews in Spark
df_reviews_spark = spark.read.parquet("../data/nyc/nyc-reviews.parquet")
df_reviews_spark.show(5)


In [ ]:

# Load reviews in DuckDB
duckdb.sql("CREATE TABLE reviews AS SELECT * FROM '../data/nyc/nyc-reviews.parquet'")
duckdb.sql("SELECT * FROM reviews LIMIT 5")


### Q16: How many reviews does each listing have?

In [ ]:

# Pandas
review_counts_pd = df_reviews_pd.groupby("listing_id").size().reset_index(name="num_reviews")
df_joined_pd = df_pd.merge(review_counts_pd, how="left", left_on="id", right_on="listing_id")
df_joined_pd[["id", "name", "num_reviews"]].head()


In [ ]:

# PySpark
from pyspark.sql.functions import count
review_counts_spark = df_reviews_spark.groupBy("listing_id").agg(count("*").alias("num_reviews"))
df_joined_spark = df_spark.join(review_counts_spark, df_spark.id == review_counts_spark.listing_id, "left")
df_joined_spark.select("id", "name", "num_reviews").show(5)


In [ ]:

# DuckDB
duckdb.sql("""
CREATE TABLE joined AS
SELECT l.*, COUNT(r.id) AS num_reviews
FROM listings l
LEFT JOIN reviews r ON l.id = r.listing_id
GROUP BY l.*
LIMIT 5
""")


### Q17: What's the average number of reviews per room type?

In [ ]:

df_joined_pd.groupby("room_type")["num_reviews"].mean()


In [ ]:

df_joined_spark.groupBy("room_type").avg("num_reviews").show()


In [ ]:

duckdb.sql("SELECT room_type, AVG(num_reviews) FROM joined GROUP BY room_type")


### Q18: What are the top 5 listings by number of reviews?

In [ ]:

df_joined_pd.sort_values("num_reviews", ascending=False)[["name", "host_name", "num_reviews"]].head()


In [ ]:

df_joined_spark.select("name", "host_name", "num_reviews").orderBy("num_reviews", ascending=False).show(5)


In [ ]:

duckdb.sql("SELECT name, host_name, num_reviews FROM joined ORDER BY num_reviews DESC LIMIT 5")
